In [1]:
import pandas as pd

### What is overlap between extracted drugs and the drugs in DrugCentral database?

In [2]:
#import drug indications from drugcentral
df_dcent = pd.read_csv('../data/input/drugcentral-contraindications.csv')

In [3]:
df_dcent.head()

,id,struct_id,concept_id,relationship_name,concept_name,umls_cui,snomed_full_name,cui_semantic_type,snomed_conceptid,id-2,doid,source,xref,id-3,identifier,id_type,struct_id-2,parent_match
0,173361,5237,21000154,contraindication,Bacterial infectious disease,C0004623,Bacterial infectious disease,T047,87628006,564836,DOID:104,UMLS_CUI,C0004623,1342674,DB11767,DRUGBANK_ID,5237,NaN
1,132838,4881,21001119,contraindication,Narcolepsy,C0027404,Narcolepsy,T047,60380001,553904,DOID:8986,UMLS_CUI,C0027404,1342174,DB09034,DRUGBANK_ID,4881,NaN
2,132852,1810,40249114,contraindication,Sjogren-Larsson syndrome,C0037231,Sjogren-Larsson syndrome,T019,111303009,555298,DOID:14501,UMLS_CUI,C0037231,1342172,DB09031,DRUGBANK_ID,1810,NaN
3,132882,4382,21001432,contraindication,Hypertensive disorder,C0020538,Hypertensive disorder,T047,38341003,567463,DOID:10763,UMLS_CUI,C0020538,1342049,DB08893,DRUGBANK_ID,4382,NaN
4,132898,4867,21002202,contraindication,Disease of liver,C0023895,Disease of liver,T047,235856003,544427,DOID:409,UMLS_CUI,C0023895,1343448,DB13762,DRUGBANK_ID,4867,NaN


In [4]:
len(df_dcent)

11889

In [5]:
df_dcent =df_dcent[['identifier', 'doid']]

In [6]:
df_dcent= df_dcent.rename(columns={'identifier':'DB_ID','doid':'DO_ID'})

In [7]:
df_dcent.DO_ID= df_dcent.DO_ID.str.replace(':','_')

In [8]:
df_dcent= df_dcent.drop_duplicates()

In [9]:
df_dcent.head()

,DB_ID,DO_ID
0,DB11767,DOID_104
1,DB09034,DOID_8986
2,DB09031,DOID_14501
3,DB08893,DOID_10763
4,DB13762,DOID_409


In [10]:
print (f"#Drugs: {len(df_dcent['DB_ID'].unique())} #Diseases: {len(df_dcent['DO_ID'].unique())} #Indications: {len(df_dcent)} in DrugCentral")

#Drugs: 1292 #Diseases: 427 #Indications: 11350 in DrugCentral


In [11]:
CATEGORY= {'Effect': 0,
'Contraindication': 1,
'Symptomatic Relief': 2,
'Treatment':3}

In [12]:
df2_full = pd.read_csv('../data/output/train_data.csv')

In [13]:
df3_full = pd.read_csv('../data/output/test_data.csv')

In [14]:
df2_full.head(2)

,Context,DB_ID,DO_ID,Drug_name,Label_ID,Section,Set_ID,disease,crowd_label,noisy_labels
0,concomitant use or use in rapid succession wit...,DB00752,DOID_0050773,TRANYLCYPROMINE,9f3ad5aa-8e9d-4512-88a5-15cfd50293f9.xml,Contraindication,NaN,paraganglioma,NaN,0
1,concomitant use or use in rapid succession wit...,DB00752,DOID_225,TRANYLCYPROMINE,9f3ad5aa-8e9d-4512-88a5-15cfd50293f9.xml,Contraindication,NaN,syndrome,NaN,0


In [15]:
df3_full.head(2)

,DB_ID,DO_ID,disease,Label_ID,Set_ID,crowd_label,Drug_name,Context,Section,noisy_labels,label
0,DB00657,DOID_10824,malignant hypertension,976f404f-4857-4bad-9a3c-9bf3d5ca9f5b.xml,c6822ce4-e741-4366-b72a-5f5146354f97,NaN,Mecamylamine,for the management of moderately severe to sev...,Indications,3,3
1,DB00401,DOID_10763,hypertension,ce733b39-7857-4538-92d6-1c68a4e4eb75.xml,a6a8a8ba-3966-4f37-a58b-44d3b05bdc4e,treatment,Nisoldipine,nisoldipine extended release tablets are indic...,Indications,3,3


In [16]:
df2= df2_full[['DB_ID' ,'DO_ID']]

In [17]:
df3= df3_full[['DB_ID' ,'DO_ID']]

In [18]:
df2= df2.append(df3)

In [19]:
df_extracted =df2.drop_duplicates()

In [20]:
len(df_extracted)

4688

In [21]:
def get_common_drugs(df1, df2, drug_column):
    merged =df1.merge(df2,on=[drug_column])
    return merged[drug_column].unique()

In [22]:
len(get_common_drugs(df_dcent, df_extracted, 'DB_ID'))

752

In [23]:
common_drugs = get_common_drugs(df_dcent, df_extracted, 'DB_ID')

In [24]:
len(df_extracted[df_extracted.DB_ID.isin(common_drugs)])

3812

In [25]:
def get_number_of_common_indications(df1, df2, drug_col, disease_col):
    merged =df1.merge(df2,on=[drug_col,disease_col])
    return len(merged)

### number of extracted relations that match with contraindications

In [26]:
get_number_of_common_indications(df_dcent, df_extracted, 'DB_ID', 'DO_ID')

369

In [27]:
#only relation with label 'contraindication'
df2 =df2_full[df2_full.noisy_labels.isin( [1]) ]

In [28]:
#only relation with label 'contraindication'
df3 = df3_full[df3_full.noisy_labels.isin( [1]) ]

In [29]:
df2= df2[['DB_ID' ,'DO_ID']]
df3= df3[['DB_ID' ,'DO_ID']]
df2= df2.append(df3)

In [30]:
df_extracted =df2.drop_duplicates()
len(df2)

10081

### number of extracted relations that correctly match with the DrrugCentral indications

In [31]:
get_number_of_common_indications(df_dcent, df_extracted, 'DB_ID', 'DO_ID')

262